# Building a FileNet Troubleshooter with IBM Granite and Ollama

_*Author - Vrunda Gadesha*_

Imagine the complexities that come with running a content platform engine for an enterprise level organization. Enterprise content management systems ( ECM ) such as IBM FileNet P8, securely store millions of critical documents, from contracts and financial records to patient files and engineering plans. For many organizations, that system is not just an  object store, but supports many use cases and acts as a highly structured, intelligent library. It comes with many features to accommodate many stakeholders as well as controls permissions  to control who can see what and how information flows through the business. It’s powerful, reliable, and essential.

But with great power comes great complexity!

These components come with their own dependencies that are time consuming to support—combing through event logs, reviewing the state of the JVM and looking at disk space. These dependencies are used to help find the root cause of issues and due to their wide use in the enterprise, they usually need to be kept operational with an SLA.


When an error occurs in a system like this, it doesn’t just say "File not found." or “timeout”. Instead, a user might be met with a cryptic error code. This code contains random string of letters and numbers like FNRCA0117E. For the person facing an error  this whether they're a new developer, a junior administrator, or someone on the support team it’s like hitting a brick wall. These cryptic codes create a few major challenges:

- The code tells you what happened, but never why.

- Finding the solution becomes a frustrating time sink.

- This creates a knowledge gap that halts productivity.

This is where Generative AI changes the game. Imagine having a seasoned expert on call, 24/7, who can instantly translate that cryptic code into plain English. But it doesn’t stop there. A well-instructed AI can weave a story around the error, suggesting common scenarios where it occurs and, most importantly, providing a clear, step-by-step plan for how to investigate and fix it. Instead of a dead end, the error becomes the starting point of a guided solution.


In this tutorial, we will build the program which will generate the full troubleshooting guide using IBM's Granite model. 

## Granite 4.0 Small model

Granite 4.0 introduces the next-generation family of open, performant and trusted Granite language models (SLMs). These models deliver greater efficiency through high performance and faster inference speeds at lower operational costs compared to similar and larger frontier models.

With a range of compact sizes, Granite 4.0 is built for flexibility to meet different hardware, deployment and cost requirements. It’s designed to scale, offering unconstrained context lengths for extended document processing and complex workflows.

Like previous Granite models, Granite 4.0 models are open source under Apache 2.0, offering extensive transparency into the training data and processes. This design equips developers to customize the models with proprietary data for increased performance. 

The Granite 4.0 family consists multiple small sizes and architectures, offering developers the ability to choose the right model for their use case, while considering specific hardware, deployment, and cost requirements. 

In this tutorial we will be using Granite 4.0-H-Micro, a 3B parameters hybrid SSM-transformers model 

## Problem statement

Let's understand this scenario: 

A junior administrator is trying to help a user and sees the error FNRCA0117E pop up. The system tells them this means "The requested action is not authorized."

This is where the real problem begins. The administrator is left with a list of frustrating questions: Which action wasn't authorized? Which user's permissions are wrong? Is the problem with the user, the document, or the folder? The error code is just a clue about the system fault. It can not provide the solution.

**With this program we can bridge this gap. It will take error code as input and, using LLM, generate a full guide that answers those follow-up questions, providing a clear path to resolving the issue.**


## Understanding Our Data

To implement this, the LLM model needs specific documentation to understand the context. It doesn't have years of FileNet experience out of the box. We give this data to LLM in a simple CSV file named `filenet-errors.csv`. 

It contains three columns:

- `ErrorCode`: The unique identifier, like FNRCA0117E.
- `ErrorName`: A short, human-readable title for the error, like "Authorization Failure".
- `ErrorMessage`: The short, official description.

## Prerequisites

For this tutorial you have to install the following dependencies and make sure to run them on your computer:

- `Ollama`: This is essential to run the Granite model on your local machine. If you don't have it, you can download it from ollama.com. Make sure the application is running in the background.

- `Python`: The code is written for Python 3.7 or newer.

- `Jupyter Notebook`: This is where we'll write and run our code. If you need to install it, you can run the command: `pip install notebook`.

## Downloading and running the Granite 4 Model

The purpose of this tutorial is to demonstrate the power of using IBM's Granite 4.0 Micro Model. In order to use that model with Ollama we need to run the command below in the terminal to download it locally to our computer

`ollama pull ibm/granite4:latest`

This command tells Ollama to pull the IBM Granite 4:Micro model and configure to be available for inferencing. Once the command is complete if you run the command

`ollama list` 

You should see the model showing as being available

## Steps

Here are the steps you can follow in the Jupyter Notebook to build the FileNet Troubleshooting Guide generator.

To make this tutorial more interactive, you'll find a special feature. Below each code block, look for an **"Explain code"** button powered by Granite. Use this button to see a detailed breakdown of each line and understand the code with more depth.

### Step 1: Install dependencies

First, we'll install the essential Python libraries that our project depends on.

In [6]:
# This command installs the necessary Python libraries.
# - ollama: The official client library to communicate with our local Ollama server.
# - pandas: A powerful library for loading and working with data from our CSV file.

!pip install -q ollama pandas

### Step 2: Test connection to the local model

Now we will run a quick connection test to confirm our notebook can communicate with the local AI model.

In [7]:
import ollama


model_name = "ibm/granite4:latest"
print(f"Sending a test message to the local model: '{model_name}'...")
print("Please wait, the first response can take a moment...")

try:
    # We send a simple chat message to the model.
    response = ollama.chat(
        model=model_name,
        messages=[
            {'role': 'user', 'content': 'In one short sentence, introduce yourself.'},
        ],
        think=False,
    )
    
    # If successful, we print the text content from the model's response.
    message_content = response['message']['content']
    print("\n Connection Successful! Model responded:")
    print(f'   "{message_content}"')

except Exception as e:
    print(f"\n An error occurred. Is Ollama running?")
    print(f"   Please ensure the Ollama application is running on your computer.")
    print(f"   Error details: {e}")

Sending a test message to the local model: 'ibm/granite4:latest'...
Please wait, the first response can take a moment...

 Connection Successful! Model responded:
   "I am an AI assistant designed to help answer your questions and provide useful information."


### Step 3: Load and prepare the data

As we can see the connection with local model is established successfully, now as a next step we will load `filenet-errors.csv` file into memory and clean the data to prevent any errors.

In [8]:
import pandas as pd

# Provide your data file path below
file_name = 'filenet-errors.csv'

try:
    df = pd.read_csv(file_name)
    
    # --- Data Cleaning Best Practice ---
    # Remove any leading/trailing whitespace from column names.
    df.columns = df.columns.str.strip()
    
    print(f"Successfully loaded and cleaned data from '{file_name}'")
    print("\nCleaned column names:", df.columns.tolist())
    
    print("\nFirst 5 rows of data:")
    # display() provides a nice, table-like output in Jupyter.
    display(df.head())

except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found.")
    print("   Please make sure the CSV file is in the same folder as this Jupyter Notebook.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded and cleaned data from 'filenet-errors.csv'

Cleaned column names: ['ErrorCode', 'ErrorName', 'ErrorMessage']

First 5 rows of data:


,ErrorCode,ErrorName,ErrorMessage
0,FNRAC1001W,WARN_TRACE_LOG_ENABLED,Server trace logging is enabled.
1,FNRAC1002W,WARN_ADDON_CANNOT_BE_UNSELECTED,The selection of this add-on feature cannot be...
2,FNRAC1001E,ERR_UNEXPECTED,An unexpected error occurred.
3,FNRAC1002E,ERR_LOGIN_AUTHENTICATION,The system cannot log you on right now.
4,FNRAC1003E,ERR_LOGIN_ACTIVITY,You are no longer logged on.


### Step 4: Craft the AI prompt template

Here, we will write the master instruction set, or 'prompt,' that tells our AI expert exactly how to behave and what to create.

Our prompt template will:

1.  **Define a Persona:** Tell the model to act as an expert IBM FileNet administrator.
2.  **Provide the Input Data:** Use placeholders (`{error_code}`, `{error_message}`, etc.) that we will fill in later with our data.
3.  **Specify the Output Format:** Instruct the model to use Markdown for headings and lists, ensuring a clean, readable output.

In [9]:
# This template is our master instruction set for the Granite model.
prompt_template = """
**Instruction:**
You are an expert IBM FileNet P8 senior system administrator. Your task is to write a detailed, step-by-step troubleshooting tutorial for a junior administrator. Use the provided error details to create a realistic use-case scenario and a clear guide.

**Input Data:**
- Error Code: {error_code}
- Error Name: {error_name}
- Error Message: {error_message}

**Required Output Format (Strictly use Markdown):**

## Troubleshooting Guide: {error_code}

### 1. Understanding the Error
Provide a brief, scenario-based explanation of what this error means for a junior administrator.

### 2. Common Scenario
Describe a realistic, day-to-day situation where this error might occur. For example: "A user from the finance team is trying to add a new invoice to the system but receives this error..."

### 3. Step-by-Step Troubleshooting
1.  **First Actionable Step:** (e.g., "Verify User Permissions in ACCE"). Explain how to do this and why it's important for this specific error.
2.  **Second Actionable Step:** (e.g., "Check Server Logs"). Specify which logs to check (e.g., p8_server_error.log) and what keywords to look for.
3.  **Third Actionable Step:** (e.g., "Review Class Definitions or Folder Security"). Explain the process clearly.

### 4. Summary of Solutions
Briefly summarize the most likely solutions to resolve the error.
"""

print("Prompt template created successfully.")

Prompt template created successfully.


### Step 5: Create the AI generator function

Now Let's create a reusable Python function that acts as the 'engine' for our tool, handling the AI generation process.

This function will:

1.  Take the details of a specific error (code, message, explanation) as input.
2.  Use our `prompt_template` to format these details into a complete set of instructions.
3.  Send the final prompt to our local `file-net-troubleshooter` model via Ollama.
4.  Receive the AI-generated response and return it as clean text.

In [10]:
import ollama

def generate_guide(error_code, error_message, error_name):
    """
    Generates a single troubleshooting guide by calling the local Ollama model.

    Args:
        error_code (str): The error code from our DataFrame.
        error_message (str): The short error message.
        explanation (str): The detailed explanation of the error.

    Returns:
        str: The AI-generated troubleshooting guide in Markdown format.
    """
    # Step 1: Fill in the placeholders in our template with the specific error data.
    full_prompt = prompt_template.format(
        error_code=error_code,
        error_name=error_name,
        error_message=error_message
    )

    # Step 2: Send the complete prompt to our custom-built local model.
    try:
        response = ollama.chat(
            model=model_name, # Our custom model name
            messages=[
                {'role': 'user', 'content': full_prompt},
            ],
            think=False,
        )
        # Step 3: Extract and return just the text content from the model's response.
        return response['message']['content']
    except Exception as e:
        error_text = f" An error occurred while communicating with the Ollama model: {e}"
        print(error_text)
        return error_text

print(" Function 'generate_guide' created successfully.")

 Function 'generate_guide' created successfully.


### Step 6: Build the final flow of the troubleshooting guide generator

Finally, we will assemble all our previous steps into the final interactive application that a user can operate.

**This code block will:**

1.  Create a continuous loop that keeps the tool running.
2.  Prompt the user to enter a FileNet error code.
3.  Search our DataFrame for the code the user entered.
4.  If the code is found, it will call our `generate_guide` function and display the detailed, AI-generated guide.
5.  If the code is not found, it will provide a helpful message and even suggest similar codes the user might have meant.
6.  Allow the user to type `exit` to quit the tool gracefully.

In [12]:
from IPython.display import display, Markdown

print("--- IBM FileNet AI Troubleshooter ---")
print("Enter a FileNet Error Code to generate a guide.")
print("Type 'exit' or 'quit' to stop the tool.")
print("-" * 35)

# This loop will run continuously until the user types 'exit' or 'quit'.
while True:
    # 1. Get input from the user.
    user_input = input("\nEnter the FileNet Error Code: ")

    # 2. Check if the user wants to exit.
    if user_input.lower() in ['exit', 'quit']:
        print("\nExiting the troubleshooter. Goodbye!")
        break

    # 3. Search for the exact error code in the DataFrame (case-insensitive).
    result_df = df[df['ErrorCode'].str.lower() == user_input.lower()]

    # 4. --- SUCCESS PATH --- If a match is found:
    if not result_df.empty:
        # Get the first matching row from our data.
        error_details = result_df.iloc[0]

        print(f"\n Found '{error_details['ErrorCode']}'. Generating guide with AI, please wait...")
        print("-" * 35)

        # Call our AI function to generate the guide.
        guide_markdown = generate_guide(
            error_code=error_details['ErrorCode'],
            error_name=error_details['ErrorName'],
            error_message=error_details['ErrorMessage']
        )

        # Display the formatted result in the notebook.
        display(Markdown(guide_markdown))
        print("-" * 35)

    # 5. --- FAILURE PATH --- If no exact match is found:
    else:
        print(f"\n Error: The code '{user_input}' was not found.")

        # BONUS: Suggest similar codes based on the user's input.
        suggestions = df[df['ErrorCode'].str.contains(user_input.upper())]
        if not suggestions.empty:
            print("\nDid you mean one of these?")
            for code in suggestions['ErrorCode'].tolist():
                print(f"- {code}")
        
        print("-" * 35)

--- IBM FileNet AI Troubleshooter ---
Enter a FileNet Error Code to generate a guide.
Type 'exit' or 'quit' to stop the tool.
-----------------------------------

 Found 'FNRAC4017E'. Generating guide with AI, please wait...
-----------------------------------


# Troubleshooting Guide: FNRAC4017E

## 1. Understanding the Error
The error code **FNRAC4017E** with the name **ERR_INVALID_START_DATE** indicates that a system operation in IBM FileNet P8 is encountering an issue because the effective start date specified for a document or task is later than its effective end date. In practical terms, this means there's a logical conflict within the data being entered: the time frame assigned to something (like an invoice period) cannot be validly defined as beginning after it ends.

### Example Scenario
Imagine you are a junior administrator in charge of managing financial documents for your company’s IBM FileNet P8 system. A user from the finance team attempts to upload a new invoice with the following details:
- **Effective Start Date:** March 15, 2024
- **Effective End Date:** March 10, 2024

When they hit “Save” or “Submit,” the system throws an error indicating that the effective start date (March 15) cannot be after the end date (March 10). This is logically impossible and therefore triggers the FNRAC4017E error.

## 2. Common Scenario
**A finance team member tries to create a new invoice document in IBM FileNet P8. They input the document's effective start and end dates but mistakenly enter an end date that precedes the start date, causing the system to throw the `FNRAC4017E` error.**

This typically occurs due to human error (typos or misinterpretation of date ranges) or when there’s a misunderstanding about how these fields should be populated.

## 3. Step-by-Step Troubleshooting

### Step 1: Verify User Permissions in ACCE
**Action:** Log into the Administration Console for Content Engine (ACCE). Navigate to the user management section and check if the finance team member has the necessary permissions to create or modify documents within the appropriate content types.

**Why It Matters:** The error might also stem from insufficient rights. Without proper access, even a correct date entry won’t allow the operation to complete successfully.

### Step 2: Check Server Logs
**Action:** Access the server logs for IBM FileNet P8. Specifically, look at `p8_server_error.log` or any other relevant log files. Search for error messages around the time of the attempted action (keyword search for “ERR_INVALID_START_DATE”).

**What to Look For:** The logs might reveal if there were additional errors related to data validation or database constraints that occurred concurrently.

### Step 3: Review Class Definitions or Folder Security
**Action:** Open the class definitions in the IBM FileNet P8 repository. Check the properties of the content type associated with invoices (e.g., `IFINVOICE` class). Verify if there are any field validations related to date ranges or constraints that could prevent a valid entry.

If the issue persists, review the folder security settings where the documents reside to ensure they allow for creation/modification by the user in question.

**Why It Matters:** Misconfigured class definitions or restrictive folder permissions can directly cause this error if they don’t accommodate the date range values being entered.

## 4. Summary of Solutions
1. **Check and Correct Date Entries:** Ensure that the effective start date is not later than the end date when entering data for documents.
2. **Verify Permissions:** Confirm that the user has adequate permissions to perform the action (creating/modifying documents).
3. **Inspect Configuration Settings:** Review class definitions, folder security settings, and any associated validation rules to ensure they allow for the intended operation.

By following these steps, junior administrators can effectively troubleshoot and resolve `FNRAC4017E` errors in IBM FileNet P8, ensuring smooth operations within their content management system.

-----------------------------------

Exiting the troubleshooter. Goodbye!


## Conclusion

This tutorial is about more than just building one program. We have learned the fundamental skills needed to create your own specialized AI assistants. We have tamed a powerful yet small Large Language Model, become a prompt engineer, and built a practical solution to a real-world business problem.

A Note on System Resources and Configuration

It’s helpful to know the kind of system this tutorial was built on and what you should consider for your own machine.

This tutorial was developed and tested on a MacBook Pro with the following specifications:
- OS: macOS Sequoia
- Memory (RAM): 32 GB
- Storage (Disk) Used for Project: Approximately 4 GB

Here's a breakdown of the resources this project will use:
- Storage: The vast majority of the disk space is used by the AI model file itself. Expect the file-net-troubleshooter model to take up about 2 GB of storage. The code and data files are very small.
- Memory (RAM): This is the most important resource for running the AI. When the model is active, it's loaded into your computer's RAM. For a smooth experience, a system with at least 16 GB of RAM is recommended. While the tool may function on a machine with 8 GB of RAM, it could be quite slow.

**Guidance for Windows Users**

For those of you working on a Windows machine, the good news is that the process is almost identical. All the tools we've used—Ollama, Python, and Jupyter—are fully compatible with Windows.

The only minor difference is in the command-line interface. Instead of using a "Terminal," you will use "Command Prompt" or "PowerShell" to run the setup commands like ollama create. All the commands themselves, and all the Python code, are exactly the same.

**What You Can Build Next**

This tutorial is more than just a FileNet tool; it’s a blueprint for powerful automation. The core components—a reliable model, a defined data source, and a smart prompt—can be adapted to countless other challenges, from parsing legal documents to generating code documentation.

Think of the next step in this tool’s evolution: an assistant that you could automatically trigger to enrich IT support tickets that could be turned into a production-ready version of that service to be deployed on an application server for the entire organization.